# Capstone Project 

# Part 1: Data download and cleaning

Let's start by importing BeautifulSoup to scrape the wikipedia page 

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
print('Libraries imported!')

Libraries imported!


Find the wikipedia page and open with BeautifulSoup

In [2]:
!wget -O filename.html en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
with open('filename.html') as html_file:
    soup= BeautifulSoup(html_file, 'lxml')

--2019-02-28 08:32:52--  http://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
Resolving en.wikipedia.org (en.wikipedia.org)... 208.80.154.224, 2620:0:861:ed1a::1
Connecting to en.wikipedia.org (en.wikipedia.org)|208.80.154.224|:80... connected.
HTTP request sent, awaiting response... 301 TLS Redirect
Location: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M [following]
--2019-02-28 08:32:53--  https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
Connecting to en.wikipedia.org (en.wikipedia.org)|208.80.154.224|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 80807 (79K) [text/html]
Saving to: ‘filename.html’

filename.html       100%[=====================>]  78.91K  --.-KB/s   in 0.05s  

2019-02-28 08:32:53 (1.48 MB/s) - ‘filename.html’ saved [80807/80807]



Parse wikipedia page for data

In [3]:
#Find the first table on page
table= soup.find('table', class_='wikitable sortable')

# Build parser
n_columns=0
n_rows=0
column_names= []

row_marker= 0 

# Find number of rows and columns
# also find columns header names 
for row in table.find_all('tr'):
    td_tags= row.find_all('td')
    if len(td_tags)>0:
        n_rows+=1
        if n_columns==0:
            n_columns== len(td_tags)
            
    #Handle column names
    th_tags= row.find_all('th')
    if len(th_tags)>0 and len(column_names)==0:
        for th in th_tags:
            column_names.append(th.get_text())

columns= column_names if len(column_names)>0 else range(0,n_columns)
df= pd.DataFrame(columns= columns, index= range(0, n_rows))
row_marker=0
for row in table.find_all('tr'):
    column_marker=0
    columns= row.find_all('td')
    for column in columns:
        df.iloc[row_marker, column_marker]= column.get_text()
        column_marker+=1
    if len(columns)>0:
        row_marker+=1

# Preview of Wikipedia postal codes as a panda DataFrame
df.head()
        
        
            

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned\n
1,M2A,Not assigned,Not assigned\n
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n


Table shape before cleaning operations

In [4]:
df.shape

(289, 3)

Only process the cells that have an assigned borough. Ignore cells with a borough that is 'Not assigned'

In [5]:
df= df.drop(df[df.Borough == 'Not assigned'].index).reset_index(drop=True)

Drop new line script(\n) from table and column header

In [6]:
df.rename(columns={'Neighbourhood\n': 'Neighbourhood'}, inplace=True)
df.replace('\n', '', regex=True, inplace=True)

df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


Merge rows with same borough using commas

In [7]:
dfgrouped= df.groupby(['Postcode', 'Borough'], as_index=False, sort=False)['Neighbourhood'].apply(','.join).reset_index(name='Neighbourhood')
dfgrouped= pd.DataFrame(dfgrouped)
dfgrouped.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Not assigned


Where neighbourhood name is missing, apply borough name

In [8]:
 dfgrouped['Neighbourhood'].loc[dfgrouped['Neighbourhood']=='Not assigned']= dfgrouped['Borough']

In [9]:
# Check if any neighbourhoods left with unassigned values
dfgrouped['Neighbourhood'].loc[dfgrouped['Neighbourhood']=='Not assigned'].count()

0

In [10]:
dfgrouped.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [11]:
dfgrouped.shape

(103, 3)

## Adding geospatial coordinates 

Let's add latitude and longitude coordinates for each neighbourhood

In [12]:
# open csv file 
!wget -O 'geospatial_data.csv' http://cocl.us/Geospatial_data
print('data downloaded!')

--2019-02-28 08:32:55--  http://cocl.us/Geospatial_data
Resolving cocl.us (cocl.us)... 169.48.113.201
Connecting to cocl.us (cocl.us)|169.48.113.201|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cocl.us/Geospatial_data [following]
--2019-02-28 08:32:55--  https://cocl.us/Geospatial_data
Connecting to cocl.us (cocl.us)|169.48.113.201|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2019-02-28 08:32:58--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 107.152.26.197
Connecting to ibm.box.com (ibm.box.com)|107.152.26.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2019-02-28 08:32:58--  https://ibm.box.com/public/static/9afzr83pps4pwf2smjjc

In [13]:
df_geo= pd.read_csv('geospatial_data.csv')
df_geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Shape of our geospatial data 

In [14]:
df_geo.shape

(103, 3)

Great! geospatial data from file matches the shape of our neighbourhood dataframe

Let's join the data from that file to our dataframe using post codes as index

In [15]:
df_geo= df_geo.rename(columns={'Postal Code': 'Postcode'})
df_geo.columns

Index(['Postcode', 'Latitude', 'Longitude'], dtype='object')

In [16]:
df2= pd.merge(dfgrouped,
            df_geo[['Postcode','Latitude', 'Longitude']],
             on='Postcode')
df2.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


In [17]:
# Check shape
df2.shape

(103, 5)

Our new dataframe has the same number of lines as our original dataframe

In [18]:
# Check for missing values
df2.isnull().any()

Postcode         False
Borough          False
Neighbourhood    False
Latitude         False
Longitude        False
dtype: bool

No missing data

<a id='Section2' > </a>

# Explore and cluster 

Before we start, let's import the dependencies we need for analysing json files, mapping and clustering 

In [19]:
import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


## Select neighbourhoods from dataframe containing the word 'Toronto'

In [20]:
df_narrowed= df2[df2['Borough'].str.contains('Toronto')].reset_index(drop=True)
df_narrowed.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
1,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M4E,East Toronto,The Beaches,43.676357,-79.293031
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


Now, let's visualize this selection using geopy

Use geopy library to get the latitude and longitude values of Toronto.

In [21]:
address= 'Toronto, Ontario'
geolocator= Nominatim()
location= geolocator.geocode(address)
latitude= location.latitude
longitude= location.longitude
print('The geographical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geographical coordinates of Toronto are 43.653963, -79.387207.


Create a map of Toronto neighbourhoods with their names as labels

In [22]:
toronto_map= folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, label in zip(df_narrowed['Latitude'],df_narrowed['Longitude'],df_narrowed['Neighbourhood']):
    label= folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat, lng],
    radius=5,
    popup=label,
    color='blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(toronto_map)

toronto_map

Define Foursquare credentials

In [23]:
# @hidden_cell
CLIENT_ID = '0RXEV5SNS5DY5OOWCMCRSWFVX52ZLTQO2SGLGDIYJ2DT23GJ' # your Foursquare ID
CLIENT_SECRET = 'OW1MMRBDTYJEKUB42MH0NRQA3H3HEUXMW24JBPXFFOPP5QVF' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 0RXEV5SNS5DY5OOWCMCRSWFVX52ZLTQO2SGLGDIYJ2DT23GJ
CLIENT_SECRET:OW1MMRBDTYJEKUB42MH0NRQA3H3HEUXMW24JBPXFFOPP5QVF


### Let's explore the first neighbourhood in our dataframe

Get the neighbourhood's name

In [24]:
df3= df_narrowed
df3.loc[0, 'Neighbourhood']

'Harbourfront,Regent Park'

Get the neighborhood's latitude and longitude values.

In [25]:
neighbourhood_latitude= df3.loc[0, 'Latitude']
neighbourhood_longitude= df3.loc[0, 'Longitude']

neighbourhood_name= df3.loc[0, 'Neighbourhood']

print('The latitude and longitude of {} are {}, {}.'.format(neighbourhood_name, 
                                                               neighbourhood_latitude, 
                                                               neighbourhood_longitude))

The latitude and longitude of Harbourfront,Regent Park are 43.6542599, -79.3606359.


### Now, let's get the top 100 venues that are in Harbourfront, Regent Park within a radius of 500 meters.

First let's create the GET request URL

In [26]:

LIMIT=100
radius=500
url= 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighbourhood_latitude, 
    neighbourhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=0RXEV5SNS5DY5OOWCMCRSWFVX52ZLTQO2SGLGDIYJ2DT23GJ&client_secret=OW1MMRBDTYJEKUB42MH0NRQA3H3HEUXMW24JBPXFFOPP5QVF&v=20180605&ll=43.6542599,-79.3606359&radius=500&limit=100'

In [27]:
results= requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c779cbef594df76b89490ec'},
 'response': {'headerLocation': 'Corktown',
  'headerFullLocation': 'Corktown, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 47,
  'suggestedBounds': {'ne': {'lat': 43.6587599045, 'lng': -79.3544279001486},
   'sw': {'lat': 43.6497598955, 'lng': -79.36684389985142}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '54ea41ad498e9a11e9e13308',
       'name': 'Roselle Desserts',
       'location': {'address': '362 King St E',
        'crossStreet': 'Trinity St',
        'lat': 43.653446723052674,
        'lng': -79.3620167174383,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.653446723052674,
          'lng': -79.3620167174383}],
        'distance': 143,
       

In [28]:
# create function to find category type
def get_category_type(row):
    try:
        categories_list= row['categories']
    except:
        categories_list= row['venue.categories']
    if len(categories_list)==0:
            return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a pandas dataframe 

In [29]:
venues= results['response']['groups'][0]['items']

nearby_venues= json_normalize(venues)

#filter columns
columns_filtered= ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']

nearby_venues= nearby_venues.loc[:, columns_filtered]
#filter the category for each row
nearby_venues['venue.categories']= nearby_venues.apply(get_category_type, axis=1)

#clean up column titles
nearby_venues.columns= [col.split(".")[-1] for col in nearby_venues.columns]
nearby_venues

,name,categories,lat,lng
0,Roselle Desserts,Bakery,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,Toronto Cooper Koo Family Cherry St YMCA Centre,Gym / Fitness Center,43.653191,-79.357947
3,Morning Glory Cafe,Breakfast Spot,43.653947,-79.361149
4,Body Blitz Spa East,Spa,43.654735,-79.359874
5,Impact Kitchen,Restaurant,43.656369,-79.356980
6,Figs Breakfast & Lunch,Breakfast Spot,43.655675,-79.364503
7,Dominion Pub and Kitchen,Pub,43.656919,-79.358967
8,Corktown Common,Park,43.655618,-79.356211
9,The Distillery Historic District,Historic Site,43.650244,-79.359323


## Explore neighbourhoods in Toronto

### Let's create a function to repeat the process to all neighbourhoods in Toronto

In [30]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
        
        nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [31]:
toronto_frame= getNearbyVenues(names= df3['Neighbourhood'], latitudes= df3['Latitude'], longitudes=df3['Longitude'])

Harbourfront,Regent Park
Ryerson,Garden District
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Adelaide,King,Richmond
Dovercourt Village,Dufferin
Harbourfront East,Toronto Islands,Union Station
Little Portugal,Trinity
The Danforth West,Riverdale
Design Exchange,Toronto Dominion Centre
Brockton,Exhibition Place,Parkdale Village
The Beaches West,India Bazaar
Commerce Court,Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North,Forest Hill West
High Park,The Junction South
North Toronto West
The Annex,North Midtown,Yorkville
Parkdale,Roncesvalles
Davisville
Harbord,University of Toronto
Runnymede,Swansea
Moore Park,Summerhill East
Chinatown,Grange Park,Kensington Market
Deer Park,Forest Hill SE,Rathnelly,South Hill,Summerhill West
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown,St. James Town
First Canadian Place,Underground city


Let's check the size of our dataframe

In [32]:
print(toronto_frame.shape)

(1695, 7)


In [33]:
toronto_frame.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Harbourfront,Regent Park",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Harbourfront,Regent Park",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Harbourfront,Regent Park",43.65426,-79.360636,Toronto Cooper Koo Family Cherry St YMCA Centre,43.653191,-79.357947,Gym / Fitness Center
3,"Harbourfront,Regent Park",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
4,"Harbourfront,Regent Park",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


Let's check how many venues are listed for each neighbourhood

In [34]:
toronto_frame.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide,King,Richmond",100,100,100,100,100,100
Berczy Park,56,56,56,56,56,56
"Brockton,Exhibition Place,Parkdale Village",19,19,19,19,19,19
Business Reply Mail Processing Centre 969 Eastern,17,17,17,17,17,17
"CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara",14,14,14,14,14,14
"Cabbagetown,St. James Town",44,44,44,44,44,44
Central Bay Street,83,83,83,83,83,83
"Chinatown,Grange Park,Kensington Market",100,100,100,100,100,100
Christie,15,15,15,15,15,15


### Let's find out how many venue category types can be curated 

In [35]:
print('there are {} uniques categories.'.format(len(toronto_frame['Venue Category'].unique()))) 
'Neighborhood'in toronto_frame['Venue Category'].values

there are 236 uniques categories.


True

In [36]:
# Set pandas viewing option
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


## Analyse each neighbourhood

In [37]:
# One hot encoding 
toronto_onehot= pd.get_dummies(toronto_frame[['Venue Category']], prefix="", prefix_sep="")
# Neighborhood exists in Dummie categories, so let's change its values to the ones in the Neighborhood dataframe
toronto_onehot['Neighborhood']= toronto_frame['Neighborhood']

#Move Neighborhood column to front of dataframe
cols=toronto_onehot.columns.tolist()
cols.insert(0, cols.pop(cols.index('Neighborhood')))
toronto_onehot=toronto_onehot.reindex(cols, axis='columns')
toronto_onehot.head()

,Neighborhood,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Harbourfront,Regent Park",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Harbourfront,Regent Park",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

Let's examine the dataframe size

In [38]:
toronto_onehot.shape

(1695, 236)

In [39]:
toronto_grouped= toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide,King,Richmond",0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.04,0.0,0.0,0.0,0.010000,0.01,0.0,0.03,0.000000,0.000000,0.0,0.000000,0.030000,0.0,0.030000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.02,0.000000,0.01,0.020000,0.000000,0.0,0.01,0.02,0.010000,0.0,0.000000,0.050000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.030000,0.010000,0.060000,0.0,0.0,0.0,0.000000,0.000000,0.020000,0.000000,0.020000,0.0,0.000000,0.0,0.0,0.0,0.01,0.01,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.01,0.0,0.000000,0.000000

Let's confirm the new size

In [40]:
toronto_grouped.shape

(38, 236)

### Let's print the top 5 venues for each neighbourhood

In [41]:
top_venues= 5

for hood in toronto_grouped['Neighborhood']:
    print('___'+ hood+'___')
    temp= toronto_grouped[toronto_grouped['Neighborhood']==hood].T.reset_index()
    temp.columns= ['Venue', 'Freq']
    
    temp= temp.iloc[1:]
    temp['Freq']= temp['Freq'].astype(float)
    temp= temp.round({'Freq': 2})
    print(temp.sort_values(by= 'Freq', ascending=False).reset_index(drop=True).head(top_venues))

___Adelaide,King,Richmond___
                 Venue  Freq
0          Coffee Shop  0.06
1                 Café  0.05
2  American Restaurant  0.04
3           Steakhouse  0.04
4      Thai Restaurant  0.04
___Berczy Park___
            Venue  Freq
0     Coffee Shop  0.07
1      Restaurant  0.05
2    Cocktail Bar  0.05
3  Farmers Market  0.04
4            Café  0.04
___Brockton,Exhibition Place,Parkdale Village___
                   Venue  Freq
0         Breakfast Spot  0.11
1                   Café  0.11
2            Coffee Shop  0.11
3  Performing Arts Venue  0.05
4              Pet Store  0.05
___Business Reply Mail Processing Centre 969 Eastern___
           Venue  Freq
0    Yoga Studio  0.06
1     Restaurant  0.06
2  Burrito Place  0.06
3        Butcher  0.06
4         Garden  0.06
___CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara___
              Venue  Freq
0    Airport Lounge  0.14
1   Airport Service  0.14
2  Airport Terminal  

### Let's put that into a pandas Dataframe

First, let's write a function to sort the venues in descending order.

In [42]:
def most_common_category(row, num_top_venues):
    row_category= row.iloc[1:]
    row_categories_sorted= row_category.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [51]:
num_top_venues= 10

indicators= ['st','nd','rd']

# create columns according to top venues.
columns=['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))    
    except:
        columns.append('{}{} Most Common Venue'.format(ind+1,'th'))

# create new dataframe
top_venues_sorted= pd.DataFrame(columns= columns)
top_venues_sorted['Neighborhood']= toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    top_venues_sorted.iloc[ind, 1:]= most_common_category(toronto_grouped.iloc[ind,:], num_top_venues)
    
top_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Café,Thai Restaurant,Steakhouse,American Restaurant,Clothing Store,Gym,Hotel,Bakery,Bar
1,Berczy Park,Coffee Shop,Restaurant,Cocktail Bar,Bakery,Farmers Market,Cheese Shop,Pub,Seafood Restaurant,Beer Bar,Italian Restaurant
2,"Brockton,Exhibition Place,Parkdale Village",Coffee Shop,Café,Breakfast Spot,Pet Store,Burrito Place,Stadium,Caribbean Restaurant,Bar,Furniture / Home Store,Italian Restaurant
3,Business Reply Mail Processing Centre 969 Eastern,Yoga Studio,Garden,Comic Shop,Pizza Place,Recording Studio,Restaurant,Butcher,Burrito Place,Skate Park,Brewery
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Airport Lounge,Airport Service,Airport Terminal,Boutique,Sculpture Garden,Airport,Airport Food Court,Airport Gate,Harbor / Marina,Plane
5,"Cabbagetown,St. James Town",Coffee Shop,Restaurant,Market,Italian Restaurant,Café,Park,Pub,Bakery,Pizza Place,Beer Store
6,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Burger Joint,Bar,Chinese Restaurant,Ice Cream Shop,Bubble Tea Shop,Thai Restaurant,Salad Place
7,"Chinatown,Grange Park,Kensington Market",Café,Bar,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Dumpling Restaurant,Bakery,Coffee Shop,Mexican Restaurant,Chinese Restaurant,Gaming Cafe
8,Christie,Café,Grocery Store,Park,Convenience Store,Coffee Shop,Baby Store,Nightclub,Restaurant,Italian Restaurant,Diner
9,Church and Wellesley,Japanese Restaurant,Coffee Shop,Sushi Restaurant,Gay Bar,Restaurant,Burger Joint,Pub,Pizza Place,Gastropub,Fast Food Restaurant


## Cluster neighborhoods

Run k-means to cluster the neighborhood into 5 clusters.

In [58]:
# set number of clusters
k_clusters= 5

toronto_grouped_clustering= toronto_grouped.drop('Neighborhood',1)

#instantiate kmeans model
kmeans_toronto= KMeans(n_clusters= k_clusters, random_state=0).fit(toronto_grouped_clustering)

#check labels
kmeans_toronto.labels_[0:20]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 1, 0, 0],
      dtype=int32)

Let's create a new dataframe data includes the top venues as well as the cluster label

In [65]:
toronto_merged= df3

toronto_merged['Cluster Label']= kmeans_toronto.labels_

toronto_merged= toronto_merged.join(top_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

toronto_merged.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636,0,Coffee Shop,Café,Park,Bakery,Theater,Breakfast Spot,Mexican Restaurant,Pub,Electronics Store,Ice Cream Shop
1,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937,0,Clothing Store,Coffee Shop,Café,Cosmetics Shop,Middle Eastern Restaurant,Plaza,Theater,Japanese Restaurant,Diner,Pizza Place
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Coffee Shop,Restaurant,Café,Hotel,Clothing Store,Cocktail Bar,Cosmetics Shop,Bakery,Park,Italian Restaurant
3,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Health Food Store,Pub,Coffee Shop,Yoga Studio,Dog Run,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,0,Coffee Shop,Restaurant,Cocktail Bar,Bakery,Farmers Market,Cheese Shop,Pub,Seafood Restaurant,Beer Bar,Italian Restaurant


Finally, let's visualize the resulting clusters

In [72]:

clusters_map= folium.Map(location=[latitude, longitude], zoom_start=11)

#set color scheme for clusters
x= np.arange(k_clusters)
ys= [(i+x+(i*x)**2)for i in range(k_clusters)]
colors_array= cm.rainbow(np.linspace(0,1, len(ys)))
rainbow= [colors.rgb2hex(i) for i in colors_array]

# place clusters on map
markers_colors=[]

for lat, long, poi, cluster in zip(toronto_merged['Latitude'],toronto_merged['Longitude'],toronto_merged['Neighbourhood'],toronto_merged['Cluster Label']):
    label= folium.Popup(str(poi)+ ' Cluster ' +str(cluster))
    folium.CircleMarker(
    [lat, long],
    radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(clusters_map)
    
clusters_map



## Examine clusters

### Cluster 1

In [75]:
toronto_merged.loc[toronto_merged['Cluster Label']==0, toronto_merged.columns[[1]+ list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,0,Coffee Shop,Café,Park,Bakery,Theater,Breakfast Spot,Mexican Restaurant,Pub,Electronics Store,Ice Cream Shop
1,Downtown Toronto,0,Clothing Store,Coffee Shop,Café,Cosmetics Shop,Middle Eastern Restaurant,Plaza,Theater,Japanese Restaurant,Diner,Pizza Place
2,Downtown Toronto,0,Coffee Shop,Restaurant,Café,Hotel,Clothing Store,Cocktail Bar,Cosmetics Shop,Bakery,Park,Italian Restaurant
3,East Toronto,0,Health Food Store,Pub,Coffee Shop,Yoga Studio,Dog Run,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space
4,Downtown Toronto,0,Coffee Shop,Restaurant,Cocktail Bar,Bakery,Farmers Market,Cheese Shop,Pub,Seafood Restaurant,Beer Bar,Italian Restaurant
5,Downtown Toronto,0,Coffee Shop,Café,Italian Restaurant,Burger Joint,Bar,Chinese Restaurant,Ice Cream Shop,Bubble Tea Shop,Thai Restaurant,Salad Place
6,Downtown Toronto,0,Café,Grocery Store,Park,Convenience Store,Coffee Shop,Baby Store,Nightclub,Restaurant,Italian Restaurant,Diner
7,Downtown Toronto,0,Coffee Shop,Café,Thai Restaurant,Steakhouse,American Restaurant,Clothing Store,Gym,Hotel,Bakery,Bar
8,West Toronto,0,Bakery,Pharmacy,Supermarket,Discount Store,Gym / Fitness Center,Bar,Fast Food Restaurant,Café,Music Venue,Middle Eastern Restaurant
9,Downtown Toronto,0,Coffee Shop,Aquarium,Hotel,Café,Pizza Place,Italian Restaurant,Scenic Lookout,Bakery,Restaurant,Brewery


### Cluster 2

In [76]:
toronto_merged.loc[toronto_merged['Cluster Label']==1, toronto_merged.columns[[1]+ list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,Central Toronto,1,Park,Swim School,Dim Sum Restaurant,Bus Line,Yoga Studio,Doner Restaurant,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant
24,West Toronto,1,Breakfast Spot,Gift Shop,Cuban Restaurant,Bank,Eastern European Restaurant,Bar,Restaurant,Italian Restaurant,Dog Run,Bookstore
27,West Toronto,1,Coffee Shop,Pizza Place,Café,Sushi Restaurant,Italian Restaurant,Bookstore,Smoothie Shop,Restaurant,Latin American Restaurant,Indie Movie Theater


### Cluster 3

In [78]:
toronto_merged.loc[toronto_merged['Cluster Label']==2, toronto_merged.columns[[1]+ list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
28,Central Toronto,2,Playground,Park,Trail,Tennis Court,Cosmetics Shop,Discount Store,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space


### Cluster 4

In [80]:
toronto_merged.loc[toronto_merged['Cluster Label']==3, toronto_merged.columns[[1]+ list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,West Toronto,3,Coffee Shop,Café,Breakfast Spot,Pet Store,Burrito Place,Stadium,Caribbean Restaurant,Bar,Furniture / Home Store,Italian Restaurant
35,Downtown Toronto,3,Coffee Shop,Café,Hotel,Restaurant,American Restaurant,Gastropub,Asian Restaurant,Gym,Bar,Steakhouse


### Cluster 5

In [82]:
toronto_merged.loc[toronto_merged['Cluster Label']==4, toronto_merged.columns[[1]+ list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Central Toronto,4,Sporting Goods Shop,Coffee Shop,Yoga Studio,Gift Shop,Fast Food Restaurant,Metro Station,Mexican Restaurant,Diner,Dessert Shop,Park
